In [ ]:
#!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [1]:
import os
import tensorflow as tf
print(tf.version.VERSION)

2.3.0


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [5]:
def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

In [6]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### SavedModel format

In [7]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1481 - sparse_categorical_accuracy: 0.6720
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4115 - sparse_categorical_accuracy: 0.8920
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2775 - sparse_categorical_accuracy: 0.9290
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2049 - sparse_categorical_accuracy: 0.9440
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1481 - sparse_categorical_accuracy: 0.9750
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model\assets


The SavedModel format is a directory containing a protobuf binary and a TensorFlow checkpoint. Inspect the saved model directory:

In [8]:
# my_model directory
# !ls saved_model
!dir saved_model

 Volume in drive D has no label.
 Volume Serial Number is 548E-A4BA

 Directory of D:\Sandbox\Github\TF_786\1. Introduction to TensorFlow\AZ\saved_model

03/21/2021  03:19 PM    <DIR>          .
03/21/2021  03:19 PM    <DIR>          ..
03/21/2021  03:19 PM    <DIR>          my_model
               0 File(s)              0 bytes
               3 Dir(s)  113,888,010,240 bytes free


In [11]:
# Contains an assets folder, saved_model.pb, and variables folder.
# !ls saved_model/my_model
!dir saved_model\my_model

 Volume in drive D has no label.
 Volume Serial Number is 548E-A4BA

 Directory of D:\Sandbox\Github\TF_786\1. Introduction to TensorFlow\AZ\saved_model\my_model

03/21/2021  03:19 PM    <DIR>          .
03/21/2021  03:19 PM    <DIR>          ..
03/21/2021  03:19 PM    <DIR>          assets
03/21/2021  03:19 PM            78,186 saved_model.pb
03/21/2021  03:19 PM    <DIR>          variables
               1 File(s)         78,186 bytes
               4 Dir(s)  113,888,010,240 bytes free


In [12]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [18]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print(f'Restored model, accuracy: {100 * acc}')

32/32 - 0s - loss: 0.4323 - sparse_categorical_accuracy: 0.8630
Restored model, accuracy: 86.2999975681305
(1000, 10)


In [19]:
print(new_model.predict(test_images).shape)

(1000, 10)


### HDF5 format

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. 

In [14]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1545 - sparse_categorical_accuracy: 0.6610
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4330 - sparse_categorical_accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2897 - sparse_categorical_accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2104 - sparse_categorical_accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1508 - sparse_categorical_accuracy: 0.9660


Now, recreate the model from that file:

In [15]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print(f'Restored model, accuracy: {100 * acc}')

32/32 - 0s - loss: 0.4323 - sparse_categorical_accuracy: 0.8630
Restored model, accuracy: 86.2999975681305
